In [1]:
# Necessary imports
import re
import emoji
import random
import pandas as pd
import numpy as np
from multiprocessing import  Pool
import time
import translators as ts

Using United States server backend.


In [2]:
# Function to remove emojis in text, since these conflict during translation
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)


def approximate_emoji_insert(string, index,char):
    if(index<(len(string)-1)):
        
        while(string[index]!=' ' ):
            if(index+1==len(string)):
                break
            index=index+1
        return string[:index] + ' '+char + ' ' + string[index:]
    else:
        return string + ' '+char + ' ' 
    


def extract_emojis(str1):
    try:
        return [(c,i) for i,c in enumerate(str1) if c in emoji.UNICODE_EMOJI]
    except AttributeError:
        return []

In [3]:
import os
if not os.path.exists('translatedtrain'):
    os.makedirs('translatedtrain')
if not os.path.exists('translatedval'):
    os.makedirs('translatedval')
if not os.path.exists('translatedtest'):
    os.makedirs('translatedtest')

In [4]:
# Use multiprocessing framework for speeding up translation process
def parallelize_dataframe(df, func, n_cores=1):
    '''parallelize the dataframe'''
    # df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    # df = pd.concat(pool.map(func, df))
    # pool.close()
    # pool.join()
    # print("done parallelizing")
    return func(df)

# Main function for translation
def translate(x,lang):
    '''provide the translation given text and the language'''
    #x=preprocess_lib.preprocess_multi(x,lang,multiple_sentences=False,stop_word_remove=False, tokenize_word=False, tokenize_sentence=False)
    # translator = Translator()
    # translator = Translator(to_lang='en',from_lang='pt')
    emoji_list=extract_emojis(x)
    # print("try")
    # translated_text = translator.translate(x, src = 'pt', dest='en')
    # translated_text = translated_text.text
    try:
        # print("try")
        # translated_text = translator.translate(x)
        # translated_text=translate_text(x,'pt','en')
        translated_text= ts.google(x, from_language='pt', to_language='en')
        # print("translated successfully")
    except:
        translated_text=x
        print("not translated")
    for ele in emoji_list:
        translated_text=approximate_emoji_insert(translated_text, ele[1],ele[0])
    # print("done translating")
    return translated_text

def add_features(df):
    '''adding new features to the dataframe'''
    translated_text=[]
    for index,row in df.iterrows():
        # if('pt' in ['en','unk']):
        #     translated_text.append(row['text'])
        # else:
        translated_text.append(translate(row['text'], 'pt'))    
    df["translated"]=translated_text
    return df

In [5]:
import glob 
train_files = glob.glob('train/*.csv')
test_files = glob.glob('test/*.csv')
val_files = glob.glob('val/*.csv')
files= train_files+test_files+val_files

In [6]:
from tqdm import tqdm_notebook
size=10

for file in files:
    wp_data=pd.read_csv(file)
    end= len(wp_data.index)
    list_df=[]
    for i in tqdm_notebook(range(0,end,size)):
                print(i,"_iteration")
                df_new=parallelize_dataframe(wp_data[i:i+size],add_features,n_cores=1)
                list_df.append(df_new)
    df_translated=pd.concat(list_df,ignore_index=True)
    file_name='translated'+file
    df_translated.to_csv(file_name,index=False)

  0%|          | 0/397 [00:00<?, ?it/s]

.........      1   
192  O que essa mulher está falando ??? Eu tô esper...      0   
193  Vale a pena ler esse texto. Ronaldo Laranjeira...      0   
194  RT @Tweet25Hora: O teu Tweet em destaque na 2ª...      0   
195  RT @ajulysantos: Cynara Menezes hoje vai sonha...      0   
196  Parabéns pra sapatão mais piranha de santaa cr...      1   
197  RT @2Souoquesou: A pessoa feia pode ficar boni...      1   
198  ESTAR ACORDADO\nSÁBADO DE MANHÃ\nISSO QUE SEPA...      1   
199  Tenho tanto medo de avião que quando ele passa...      0   

                                            translated  
190  After two days of very productive conversation...  
191               I miss the dictatorship ............  
192  What is this woman talking about ???I'm waitin...  
193  It's worth reading this text.Ronaldo Laranjeir...  
194  RT @ tweet25hora: your tweet highlighted in th...  
195  RT @AJULYSANTOS: Cynara Menezes today will dre...  
196  Congratulations to Sapatão more Piranha de San...  
197

In [85]:
import translators as ts

Using United States server backend.


In [98]:
ts.google(word, from_language='pt')

'I lived in Maranho 8 to'